# Healthcare Graphs

## Datasets
- [Life Expectancy](https://tablebuilder.singstat.gov.sg/table/TS/M810501#!)
- [CPI](https://data.gov.sg/datasets/d_de7e93a1d0e22c790516a632747bf7f0/view?dataExplorerPage=9)

In [52]:
import pandas as pd
import requests

### Global variables 

In [53]:
BASE_DATA_GOV_URL = "https://data.gov.sg/api/action/datastore_search"
BASE_SINGSTAT_URL = "https://tablebuilder.singstat.gov.sg/api/table/tabledata"

## Retrieve data

In [54]:
# function to fetch data from data.gov.sg 
def fetch_datagov_dataset(dataset_id: str, limit: int=10_000_000) -> pd.DataFrame:
    response = requests.get(BASE_DATA_GOV_URL, params={"resource_id": dataset_id, "limit": limit})
    data = response.json()
    if not data["success"]:
        raise Error(f"Failed to fetch dataset ({dataset_id}).")
    return pd.DataFrame(data["result"]["records"])

In [55]:
def parse_singstat_2d_data(raw_data: dict) -> pd.DataFrame:
    rows = raw_data.get("Data", {}).get("row", [])
    records = []

    for row in rows:
        series_name = row.get("rowText", "")
        columns = row.get("columns", [])
        series_data = {col["key"]: col["value"] for col in columns}
        series_data["Series"] = series_name
        records.append(series_data)

    df = pd.DataFrame(records)
    df.set_index("Series", inplace=True)
    df.columns.name = "Year"
    return df

def fetch_singstat_dataset(dataset_id: str, limit: int = 10_000_000) -> pd.DataFrame:
    response = requests.get(
        f"{BASE_SINGSTAT_URL}/{dataset_id}",
        params={"limit": limit},
        headers={
            "Accept": "application/json",
            "User-Agent": "curl/8.11.1", # Need to fake the user agent because SingStat blocks `python-requests`.
        }
    )
    data = response.json()
    return parse_singstat_2d_data(data)

In [56]:
# fetch CPI dataset from data.gov.sg 
datagov_dataset_id = "d_de7e93a1d0e22c790516a632747bf7f0"
cpi_df = fetch_datagov_dataset(datagov_dataset_id)

sing_stat_dataset_id = "M810501"
lx_df = fetch_singstat_dataset(sing_stat_dataset_id)

In [57]:
cpi_df.head()

,_id,DataSeries,2024Dec,2024Nov,2024Oct,2024Sep,2024Aug,2024Jul,2024Jun,2024May,...,1961Oct,1961Sep,1961Aug,1961Jul,1961Jun,1961May,1961Apr,1961Mar,1961Feb,1961Jan
0,1,All Items,117.197,116.792,116.756,117.123,116.829,116.028,116.323,116.574,...,24.463,24.393,24.517,24.276,24.223,24.053,24.187,24.585,24.565,24.542
1,2,Food,119.272,118.981,118.897,118.693,118.553,118.368,118.361,118.332,...,21.614,21.506,21.699,21.323,21.242,20.967,21.172,21.82,21.777,21.762
2,3,Food Excl Food Serving Services,118.209,117.746,117.707,117.649,117.03,116.54,116.641,116.975,...,na,na,na,na,na,na,na,na,na,na
3,4,Bread & Cereals,119.91,119.717,119.428,118.818,118.12,118.209,117.849,117.711,...,na,na,na,na,na,na,na,na,na,na
4,5,Rice,105.233,104.995,104.408,105.534,103.426,104.208,103.349,104.059,...,na,na,na,na,na,na,na,na,na,na


In [58]:
lx_df.head()

Year,1957,1960,1965,1970,1975,1980,1981,1982,1983,1984,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Series,,,,,,,,,,,,,,,,,,,,,
Total Life Expectancy At Birth (Residents),61.1,62.9,64.5,65.8,66.8,72.1,72.5,72.6,73,73.3,...,82.6,82.9,83,83.2,83.4,83.7,83.7,83.2,83,83
Male Life Expectancy At Birth (Residents),59.4,61.2,62.8,64.1,65.1,69.8,70.1,70.2,70.5,70.9,...,80.3,80.5,80.7,80.9,81.2,81.4,81.3,80.8,80.7,80.7
Female Life Expectancy At Birth (Residents),63.2,65,66.6,67.8,68.8,74.7,75.2,75.3,75.7,75.8,...,84.8,85.1,85.1,85.4,85.5,85.9,85.9,85.5,85.2,85.2
Total Life Expectancy At Age 65 Years (Residents),8.1,8.3,8.3,8.4,8.5,14,14.2,14.2,14.4,14.6,...,20.6,20.8,20.8,21,21.2,21.4,21.3,20.9,20.7,20.7
Male Life Expectancy At Age 65 Years (Residents),7.5,7.8,7.8,8,8.2,12.6,12.7,12.8,13,13.2,...,18.8,18.9,19,19.2,19.4,19.5,19.5,19.1,19,19
